In [ ]:
%load_ext autoreload
%autoreload 2
%cd ..

In [ ]:
import numpy as np
import pandas as pd
import scipy
import toolkit as ftk
import statsmodels.api as sm
from sklearn import linear_model

unit_price = pd.Series([100,100.3,102.91,104.04,103.1,104.55,107.06,108.66,115.83,114.21,118.67,118.07,127.64,132.74,127.83,120.03,121.71,115.75,113.32,120.34,127.32,119.18,121.2,120.72,120.48,117.95,119.24,124.85,127.84,132.06,131.14,137.3,138.13,139.51,139.23,143.96,145.4],
    index=pd.date_range('1999-12', periods=37, freq='M'))
benchmark_price = pd.Series([100,100.2,102.71,104.55,103.4,104.85,107.26,108.76,115.83,114.1,118.89,118.53,128.37,133.38,128.31,120.35,122.16,116.29,113.97,120.81,127.57,119.03,121.29,120.92,120.8,117.66,118.48,123.58,127.16,132,131.73,138.45,140.39,142.21,142.64,147.49,150.59],
    index=pd.date_range('1999-12', periods=37, freq='M'))
risk_free_return = pd.Series([0.001,0.001,0.002,0.002,0.002,0.002,0.002,0.003,0.003,0.004,0.004,0.003,0.003,0.003,0.003,0.004,0.002,0.002,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.002,0.002,0.002,0.002,0.002,0.002,0.002],
    index=pd.period_range('2000-01', periods=36, freq='M'))

aa = ftk.price_to_return(unit_price)
bb = ftk.price_to_return(benchmark_price)

prices = pd.concat([unit_price, benchmark_price], axis=1)
benchmarks = prices = pd.concat([benchmark_price, unit_price], axis=1)

x = pd.Series(np.arange(1, 11), index= pd.date_range('2023-10-27', periods=10, freq='D'))
x = x[x.index.day_of_week < 5]

ftk.summary(x)

In [1]:
# https://www.visualcapitalist.com/historical-returns-by-asset-class/
import toolkit as ftk
tickers = {'VFIAX': 'US Large Cap', 'VSMAX': 'US Small Cap', 'VTMGX': 'DM Eq', 'VEMAX': 'EM Eq', 'VBTLX': 'US IG', 'VWEAX': 'US HY', 'VTABX': 'DM IG', 'VGSLX': 'REITs', 'IAU': 'Gold'}
px = ftk.get_yahoo_bulk(tickers.keys()).rename(columns=tickers)
returns = px.resample('M').last().pct_change().dropna().to_period('M')

[*********************100%%**********************]  9 of 9 completed


In [177]:
import numpy as np
import pandas as pd
from scipy import optimize
import toolkit as ftk

# Constant rebalance
def portfolio_volatility(weights: np.ndarray, cov: pd.DataFrame) -> float:
    return np.sqrt(weights @ cov @ weights)

def portfolio_return(weights: np.ndarray, er: pd.Series) -> float:
    # Identical to (er * weights).sum()
    return weights @ er

def portfolio_returns(weights: np.ndarray, returns: pd.DataFrame) -> pd.Series:
    # Identical to (returns * weights).sum(axis=1)
    return returns @ weights

# Weights
def equal_weight(er: pd.Series) -> np.ndarray:
    n = len(er)
    return np.repeat(1 / n, n)

def max_return(er: pd.Series) -> np.ndarray:
    wtg = pd.Series(0, index=er.index)
    wtg.loc[er.idxmax()] = 1
    return wtg

def max_sharpe(er: pd.Series, cov: pd.DataFrame, rfr: float = 0., min: float = float('-inf'), max: float = float('inf')) -> np.ndarray:

    def neg_sharpe(weights, er, cov, rfr):
        return (rfr - portfolio_return(weights, er)) / portfolio_volatility(weights, cov)
    
    n = len(er)
    fully_invest = {'type': 'eq',
                    'fun': lambda weights: np.sum(weights) - 1}
    wtg = optimize.minimize(neg_sharpe,
                            equal_weight(er),
                            args=(er, cov, rfr),
                            method='SLSQP',
                            options={'disp': False},
                            constraints=(fully_invest,),
                            bounds=((min, max),) * n
                            )
    return wtg.x

def min_vol(er: pd.Series, cov: pd.DataFrame, min: float = float('-inf'), max: float = float('inf')) -> np.ndarray:
    # np.ones_like(d) doesn't work
    return max_sharpe(np.repeat(1., len(er)), cov, rfr=0, min=min, max=max)

def min_vol_at(target: float, er: pd.Series, cov: pd.DataFrame, min: float = float('-inf'), max: float = float('inf')) -> np.ndarray:
    n = len(er)
    fully_invest = {'type': 'eq',
                    'fun': lambda weights: np.sum(weights) - 1}
    target_return = {'type': 'eq',
                     'fun': lambda weights: portfolio_return(weights, er) - target}
    wtg = optimize.minimize(portfolio_volatility,
                            equal_weight(er),
                            args=(cov),
                            method='SLSQP',
                            options={'disp': False},
                            constraints=(target_return, fully_invest,),
                            bounds=((min, max),) * n
                            )
    return wtg.x

def inverse_vol(d: pd.DataFrame) -> np.ndarray:
    inverse = 1 / ftk.volatility(d)
    return (inverse / inverse.sum()).to_numpy()

def risk_parity(d: pd.DataFrame) -> np.ndarray:
    pass

cov = ftk.covariance(returns, annualize=True)
er = ftk.compound_return(returns, annualize=True)
max_sharpe(er, cov)

def debug(name, w):
    print(name, portfolio_return(w, er), portfolio_volatility(w, cov), portfolio_return(w, er)/portfolio_volatility(w, cov), w)

debug('Eq', equal_weight(er))
debug('Max. Sharpe', max_sharpe(er, cov))
debug('Inv. Vol', inverse_vol(cov))
debug('Min. Vol', min_vol(er, cov))
debug('Min. Vol At', min_vol_at(0.1, er, cov))
debug('Min. Vol At', min_vol_at(0.1, er, cov, 0.05, 0.2))

Eq 0.04617064345934074 0.09682066433218756 0.47686765813681303 [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
Max. Sharpe 0.15006784379994337 0.10868525020944544 1.3807562986766857 [ 0.20777951 -0.52739586 -0.07899392  1.7096261  -0.30272308 -0.46855735
  0.60047041 -0.78301938  0.64281356]
Inv. Vol 0.022834636694604487 0.049366064429871764 0.4625573652330098 [0.04331821 0.34386663 0.02982394 0.02830121 0.02852559 0.0217748
 0.39445201 0.02869562 0.08124199]
Min. Vol 0.015449296097601416 0.03486816863505781 0.44307735973457735 [-0.0351568   0.26335312  0.00576763 -0.0010358  -0.14193958  0.03594583
  0.70539924 -0.03605235  0.20371871]
Min. Vol At 0.1000000000023461 0.07345738099035791 1.3613335876414134 [ 0.11738549 -0.23689044 -0.04666476  1.07346661 -0.24300713 -0.28082629
  0.64449326 -0.50594749  0.47799076]
Min. Vol At 0.06138931551793917 0.11544702418226692 0.5317531218562911 [0.15 0.05 0.05 0.2  0.05 0.2  0.05 0.2  0.05]


In [272]:
w0 = min_vol(er, cov)
w1 = max_return(er)
y = np.linspace(portfolio_return(w0, er), portfolio_return(w1, er), 30)
ef = [min_vol_at(yy, er, cov) for yy in y]
ns = [min_vol_at(yy, er, cov, 0) for yy in y]
nl = [min_vol_at(yy, er, cov, 0, 1) for yy in y]
x_ef = [portfolio_volatility(w, cov) for w in ef]
x_ns = [portfolio_volatility(w, cov) for w in ns]
x_nl = [portfolio_volatility(w, cov) for w in nl]


In [273]:
source = pd.DataFrame({
    'ef': x_ef,
    'ns': x_ns,
    'nl': x_nl,
    'y': list(y)
})

In [274]:
source
aa = source.melt(id_vars=['y'], value_name='x', var_name='method')

import altair as alt
alt.Chart(aa).mark_line(color='black').encode(
    x='x',
    y='y',
    color='method',
)

c:\Users\Admin\OneDrive\Python\financial-toolkit\.venv\Lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)